# Data Cleaning - Tech Industry Salaries

This notebook handles the cleaning and preprocessing of the raw salary dataset.

## Objectives:
- Load raw data
- Handle missing values
- Standardize column names
- Remove duplicates
- Create derived features
- Export cleaned data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Load Raw Data

In [ ]:
# Load the raw dataset
df_raw = pd.read_csv('../data/raw/ds_salaries.csv')

print(f"Dataset shape: {df_raw.shape}")
print(f"\nColumns: {df_raw.columns.tolist()}")
df_raw.head()

## 2. Initial Data Exploration

In [ ]:
# Check data types and missing values
print("Data Types and Missing Values:")
print("=" * 50)
df_info = pd.DataFrame({
    'Column': df_raw.columns,
    'Data Type': df_raw.dtypes,
    'Non-Null Count': df_raw.count(),
    'Null Count': df_raw.isnull().sum(),
    'Null %': (df_raw.isnull().sum() / len(df_raw) * 100).round(2)
})
print(df_info.to_string())

In [ ]:
# Check for duplicates
duplicates = df_raw.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicates}")

if duplicates > 0:
    print("\nDuplicate rows:")
    print(df_raw[df_raw.duplicated(keep=False)].sort_values(by=df_raw.columns.tolist()))

In [ ]:
# Basic statistics
print("\nBasic Statistics:")
print("=" * 50)
df_raw.describe()

## 3. Data Cleaning Steps

In [ ]:
# Create a copy for cleaning
df_clean = df_raw.copy()

# Drop the unnamed index column if it exists
if 'Unnamed: 0' in df_clean.columns:
    df_clean = df_clean.drop('Unnamed: 0', axis=1)
    print("✓ Dropped 'Unnamed: 0' column")

In [ ]:
# Standardize column names (lowercase, underscores)
df_clean.columns = df_clean.columns.str.lower().str.replace(' ', '_')
print("✓ Standardized column names")
print(f"New columns: {df_clean.columns.tolist()}")

In [ ]:
# Standardize job titles (uppercase for consistency)
df_clean['job_title'] = df_clean['job_title'].str.upper()
print("✓ Standardized job titles to uppercase")
print(f"\nUnique job titles: {df_clean['job_title'].nunique()}")

In [ ]:
# Check for outliers in salary_in_usd
Q1 = df_clean['salary_in_usd'].quantile(0.25)
Q3 = df_clean['salary_in_usd'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

outliers = df_clean[(df_clean['salary_in_usd'] < lower_bound) | 
                     (df_clean['salary_in_usd'] > upper_bound)]

print(f"Salary outliers detected: {len(outliers)}")
print(f"Lower bound: ${lower_bound:,.0f}")
print(f"Upper bound: ${upper_bound:,.0f}")

if len(outliers) > 0:
    print("\nOutlier examples:")
    print(outliers[['job_title', 'experience_level', 'salary_in_usd', 'company_location']].head(10))

In [ ]:
# Remove extreme outliers (optional - adjust based on domain knowledge)
# For now, we'll flag but keep them for transparency
df_clean['is_outlier'] = ((df_clean['salary_in_usd'] < lower_bound) | 
                           (df_clean['salary_in_usd'] > upper_bound)).astype(int)

print(f"✓ Flagged {df_clean['is_outlier'].sum()} outliers")

## 4. Feature Engineering

In [ ]:
# Create seniority level mapping
seniority_map = {
    'EN': 'JUNIOR',
    'MI': 'MID',
    'SE': 'SENIOR',
    'EX': 'EXECUTIVE'
}

df_clean['seniority'] = df_clean['experience_level'].map(seniority_map)
print("✓ Created 'seniority' column")
print(f"\nSeniority distribution:")
print(df_clean['seniority'].value_counts())

In [ ]:
# Create remote work category
def categorize_remote(ratio):
    if ratio == 0:
        return 'On-site'
    elif ratio == 50:
        return 'Hybrid'
    elif ratio == 100:
        return 'Remote'
    else:
        return 'Other'

df_clean['work_setting'] = df_clean['remote_ratio'].apply(categorize_remote)
print("✓ Created 'work_setting' column")
print(f"\nWork setting distribution:")
print(df_clean['work_setting'].value_counts())

In [ ]:
# Create company size label
size_map = {
    'S': 'Small',
    'M': 'Medium',
    'L': 'Large'
}

df_clean['company_size_label'] = df_clean['company_size'].map(size_map)
print("✓ Created 'company_size_label' column")
print(f"\nCompany size distribution:")
print(df_clean['company_size_label'].value_counts())

In [ ]:
# Create job category based on job title
def categorize_job(title):
    title = title.upper()
    if 'DATA SCIENTIST' in title or 'DATA SCIENCE' in title:
        return 'Data Science'
    elif 'DATA ENGINEER' in title or 'DATA ENGINEERING' in title:
        return 'Data Engineering'
    elif 'DATA ANALYST' in title or 'ANALYST' in title:
        return 'Data Analytics'
    elif 'MACHINE LEARNING' in title or 'ML ENGINEER' in title or 'ML ' in title:
        return 'Machine Learning'
    elif 'DIRECTOR' in title or 'HEAD OF' in title or 'MANAGER' in title:
        return 'Management'
    elif 'RESEARCH' in title:
        return 'Research'
    elif 'AI' in title or 'ARTIFICIAL INTELLIGENCE' in title:
        return 'AI/ML'
    else:
        return 'Other'

df_clean['job_category'] = df_clean['job_title'].apply(categorize_job)
print("✓ Created 'job_category' column")
print(f"\nJob category distribution:")
print(df_clean['job_category'].value_counts())

In [ ]:
# Create salary bands
def salary_band(salary):
    if salary < 50000:
        return '<50K'
    elif salary < 100000:
        return '50K-100K'
    elif salary < 150000:
        return '100K-150K'
    elif salary < 200000:
        return '150K-200K'
    else:
        return '200K+'

df_clean['salary_band'] = df_clean['salary_in_usd'].apply(salary_band)
print("✓ Created 'salary_band' column")
print(f"\nSalary band distribution:")
print(df_clean['salary_band'].value_counts().sort_index())

## 5. Data Quality Checks

In [ ]:
# Final check for missing values
print("Missing values in cleaned dataset:")
print("=" * 50)
missing = df_clean.isnull().sum()
missing_pct = (missing / len(df_clean) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

if missing.sum() == 0:
    print("\n✓ No missing values found!")

In [ ]:
# Check data types
print("\nData types:")
print("=" * 50)
print(df_clean.dtypes)

In [ ]:
# Summary statistics for cleaned data
print("\nSummary of cleaned dataset:")
print("=" * 50)
print(f"Total records: {len(df_clean):,}")
print(f"Total columns: {len(df_clean.columns)}")
print(f"Years covered: {df_clean['work_year'].min()} - {df_clean['work_year'].max()}")
print(f"Countries: {df_clean['company_location'].nunique()}")
print(f"Job titles: {df_clean['job_title'].nunique()}")
print(f"\nSalary range (USD): ${df_clean['salary_in_usd'].min():,.0f} - ${df_clean['salary_in_usd'].max():,.0f}")
print(f"Mean salary (USD): ${df_clean['salary_in_usd'].mean():,.0f}")
print(f"Median salary (USD): ${df_clean['salary_in_usd'].median():,.0f}")

## 6. Export Cleaned Data

In [ ]:
# Save cleaned data
output_path = '../data/cleaned/ds_salaries_cleaned.csv'
df_clean.to_csv(output_path, index=False)
print(f"✓ Cleaned data saved to: {output_path}")
print(f"\nFinal dataset shape: {df_clean.shape}")

In [ ]:
# Display sample of cleaned data
print("\nSample of cleaned data:")
df_clean.head(10)

In [ ]:
# Column overview
print("\nFinal columns:")
print("=" * 50)
for i, col in enumerate(df_clean.columns, 1):
    print(f"{i:2d}. {col}")

## Summary

### Cleaning Steps Completed:
1. ✓ Loaded raw data
2. ✓ Removed unnecessary index column
3. ✓ Standardized column names
4. ✓ Standardized job titles
5. ✓ Identified and flagged outliers
6. ✓ Created derived features:
   - seniority (readable experience level)
   - work_setting (remote/hybrid/on-site)
   - company_size_label (readable size)
   - job_category (grouped job types)
   - salary_band (salary ranges)
7. ✓ Performed data quality checks
8. ✓ Exported cleaned dataset

### Next Steps:
- Proceed to `02_analysis.ipynb` for statistical analysis
- Explore salary trends by role, location, and experience
- Compare remote vs on-site compensation